In [32]:
from datasets import load_dataset
import dspy
import openai
import os
import re
import pandas as pd
import json
import random
from dotenv import load_dotenv
import glob


In [ ]:
load_dotenv()



In [34]:
openai.api_key = os.getenv('OPENAI_API_KEY')
openai.organization = os.getenv('OPENAI_ORGANIZATION')


In [35]:
lm = dspy.LM('openrouter/anthropic/claude-3.5-sonnet', temperature=0, max_tokens=1024)
dspy.configure(lm=lm)

In [36]:
# ds = pd.read_json('data/sa/test.json', lines= True)
# ds = ds.to_dict('records')
# ds = load_dataset('stanfordnlp/sst2')['validation']
ds = pd.read_json('../preprocessing/train_dev_test_data/sa/test.json')
ds = ds.to_dict('records')



In [ ]:
ds

In [38]:
def remove_space(text):
    """Clean up spacing and formatting in dialogue text."""
    lines = text.split('\n')
    cleaned_lines = []
    
    for line in lines:
        # Remove multiple spaces
        cleaned = ' '.join(line.split())
        
        # Fix spacing around punctuation
        cleaned = re.sub(r'\s+([.,!?:;])', r'\1', cleaned)
        cleaned = re.sub(r'([.,!?:;])\s+', r'\1 ', cleaned)
        
        # Fix contractions
        cleaned = re.sub(r'\s*\'\s*s\b', "'s", cleaned)
        cleaned = re.sub(r'\s*n\s*\'\s*t\b', "n't", cleaned)
        cleaned = re.sub(r'\s*\'\s*ve\b', "'ve", cleaned)
        cleaned = re.sub(r'\s*\'\s*re\b', "'re", cleaned)
        cleaned = re.sub(r'\s*\'\s*ll\b', "'ll", cleaned)
        cleaned = re.sub(r'\s*\'\s*d\b', "'d", cleaned)
        cleaned = re.sub(r'\s*\'\s*m\b', "'m", cleaned)
        
        # Fix spaces around parentheses
        cleaned = re.sub(r'\(\s+', '(', cleaned)
        cleaned = re.sub(r'\s+\)', ')', cleaned)
        
        # Remove leading/trailing whitespace
        cleaned = cleaned.strip()
        
        cleaned_lines.append(cleaned)
        
    return '\n'.join(cleaned_lines)


In [14]:
examples = [
    dspy.Example({ 
                  "text": remove_space(r["sentence"]), 
                  "label": r["label"]}
                  
                  ).with_inputs("text") 
    for r in ds
    ]


In [ ]:
len(examples)

In [ ]:
example = examples[835]
for k, v in example.items():
    print(f"\n{k.upper()}:\n")
    print(v)


In [39]:
def extract_prediction(text):
    matches = re.findall(r'\b[0-2]\b', text)
    # print(matches)
    parsed_answer = matches[-1] if matches else ""
    return parsed_answer

In [40]:
def eval_metric(true, prediction, trace=None):
    pred = prediction.label
    matches = re.findall(r'\b[0-2]\b', pred)
    parsed_answer = matches[-1] if matches else ""
    # print(parsed_answer)
    return parsed_answer == str(true.label)

# Evaluate the original test set

In [41]:
from dspy.evaluate import Evaluate

evaluate = Evaluate(devset= examples, metric=eval_metric, num_threads=6, display_progress=True, display_table=10)


In [20]:
class Sentiment(dspy.Signature):
    """Classify sentiment of the given text. Answer with 1 for positive, 0 for negative."""
    text = dspy.InputField()
    label = dspy.OutputField(prefix = 'Answer:')

In [21]:
class SimpleSentiment(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.Predict(Sentiment)

    def forward(self, text):
        return self.prog(text=text)


In [22]:
simple_sentiment = SimpleSentiment()

In [ ]:
pred = simple_sentiment(text=example.text)
print("\nQUESTION:\n")
print(example.text)
# print("\nANSWER:\n")
# print(example.label)
print("\nPREDICTION:\n")
print(pred)


In [ ]:
eval_metric(example, pred)

In [ ]:
evaluate = Evaluate(devset= examples, metric=eval_metric, num_threads=6, display_progress=True, display_table=10, return_outputs= True, return_all_scores=True)
results = evaluate(simple_sentiment)
items = []
for sample in results[1]:
    item = {}
    sentence = sample[0]['text']
    label = sample[0]['label']
    pred = sample[1]['label']
    item['text'] = sentence
    item['label'] = label
    item['pred'] = pred
    items.append(item)
df_result = pd.DataFrame(data = items)
df_result.to_csv('results/sa/claude-3-5-sonnet-0shot-sst2.csv')

In [29]:
class CoTSentiment(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(Sentiment)

    def forward(self, text):
        return self.prog(text=text)


In [ ]:
cot_sentiment = CoTSentiment()
pred = cot_sentiment(text=example.text)
print("\nQUESTION:\n")
print(example.text)
# print("\nANSWER:\n")
# print(example.label)
print("\nPREDICTION:\n")
print(pred)


In [ ]:
lm.inspect_history()

In [ ]:
evaluate = Evaluate(devset= examples, metric=eval_metric, num_threads=6, display_progress=True, display_table=10, return_outputs= True, return_all_scores=True)
results = evaluate(cot_sentiment)
items = []
for sample in results[1]:
    item = {}
    sentence = sample[0]['text']
    label = sample[0]['label']
    pred = sample[1]['label']
    item['text'] = sentence
    item['label'] = label
    item['pred'] = pred
    items.append(item)
df_result = pd.DataFrame(data = items)
df_result.to_csv('claude-3-5-sonnet-0shot-cot-sst2.csv')

# Evaluate by modification

## Without label change

In [41]:
def evaluate_modified_set(ds, program):
    examples = [
    dspy.Example({ 
                  "text" : remove_space(r['modified_text']), 
                  "original_text": remove_space(r['original_text']),
                  "label": int(r['label']),
                  "modified_label": int(r['label'])
                }
                  ).with_inputs("text") 
    for r in ds
    ]
    evaluate = Evaluate(devset= examples, metric=eval_metric, num_threads=6, display_progress=True, display_table=1, return_outputs= True, return_all_scores=True)
    results = evaluate(program)
    return results

In [42]:
class Sentiment(dspy.Signature):
    """Classify sentiment of the given text. Answer with 1 for positive, 0 for negative."""
    text = dspy.InputField()
    label = dspy.OutputField(prefix = 'Answer:')
class SimpleSentiment(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.Predict(Sentiment)

    def forward(self, text):
        return self.prog(text=text)
simple_sentiment = SimpleSentiment()

In [ ]:

# Configure GPT-4 as the language model
original_pred_ds = pd.read_csv('results/sa/claude-3-5-sonnet-0shot-sst2.csv', index_col=False)
original_pred_ds['text'] = original_pred_ds['text'].apply(lambda x: remove_space(x).encode('utf-8').decode('unicode-escape'))

# Get all json files in the specified directory
json_files = glob.glob('../data/modified_data/sa/*_100.json')

for json_file in json_files:
    # Load the json file
    print(json_file)
    if 'grammatical_role' in json_file or 'negation' in json_file:
        continue
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    results = evaluate_modified_set(data,simple_sentiment)

    
    # Convert results to dataframe
    items = []
    for sample in results[1]:
        item = {}
        sentence = sample[0]['text']
        label = sample[0]['label'] 
        pred = sample[1]['label']
        item['text'] = sentence
        item['modified_label'] = label
        pred = extract_prediction(pred)
        item['modified_pred'] = pred
        # original_text = sample[0]['original_text']
        original_text = sample[0]['original_text'].encode('utf-8').decode('unicode-escape')
        # original_text = r"{}".format(original_text)
        # print(original_text)
        item['original_label'] = sample[0]['label']
        item['original_text'] = original_text
        item['original_pred'] = original_pred_ds.loc[original_pred_ds['text'] == original_text]['pred'].values[0]
        items.append(item)
    
    df_result = pd.DataFrame(data=items)
    
    # Save results with filename based on input json
    output_filename = f"results/sa/claude-3-5-sonnet-0shot-{json_file.split('/')[-1].replace('.json', '')}.csv"
    df_result.to_csv(output_filename)


## With label change

In [54]:
def evaluate_modified_set(ds, program):
    examples = [
    dspy.Example({ 
                  "text" : remove_space(r['modified_text']), 
                  "original_text": remove_space(r['original_text']),
                  "label": int(r['modified_label']) if r.get('modified_label') else int(r['label']),
                  "original_label": int(r['label']),
                  "index": r['index'],
                  "type": r['type']
                }
                  ).with_inputs("text") 
    for r in ds
    ]
    evaluate = Evaluate(devset= examples, metric=eval_metric, num_threads=6, display_progress=True, display_table=1, return_outputs= True, return_all_scores=True)
    results = evaluate(program)
    return results

In [55]:
class Sentiment(dspy.Signature):
    """Classify sentiment of the given text. Answer with 1 for positive, 0 for negative."""
    text = dspy.InputField()
    label = dspy.OutputField(prefix = 'Answer:')
class SimpleSentiment(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.Predict(Sentiment)

    def forward(self, text):
        return self.prog(text=text)
simple_sentiment = SimpleSentiment()

In [ ]:
original_pred_ds = pd.read_csv('results/sa/claude-3-5-sonnet-0shot-sst2.csv', index_col=False)
original_pred_ds['text'] = original_pred_ds['text'].apply(lambda x: remove_space(x).encode('utf-8').decode('unicode-escape'))

# Get all json files in the specified directory
json_files = glob.glob('../data/modified_data/sa/*_100.json')

for json_file in json_files:
    # Load the json file
    print(json_file)
    if not any(x in json_file for x in ['negation']):
        continue
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    results = evaluate_modified_set(data,simple_sentiment)

    
    # Convert results to dataframe
    items = []
    for sample in results[1]:
        item = {}
        sentence = sample[0]['text']
        pred = sample[1]['label']
        item['text'] = sentence
        item['modified_label'] = sample[0]['label'] 
        pred = extract_prediction(pred)
        item['modified_pred'] = pred
        # original_text = sample[0]['original_text']
        # print(original_text)
        original_text = sample[0]['original_text'].encode('utf-8').decode('unicode-escape')

        item['original_label'] = sample[0]['original_label']
        item['original_text'] = original_text
        item['original_pred'] = original_pred_ds.iloc[sample[0]['index']]['pred']
        item['type'] = sample[0]['type']
        items.append(item)
    
    df_result = pd.DataFrame(data=items)
    
    
    # Save results with filename based on input json
    output_filename = f"results/sa/claude-3-5-sonnet-0shot-{json_file.split('/')[-1].replace('.json', '')}.csv"
    df_result.to_csv(output_filename)


# Aggregate results

In [57]:
from scipy import stats

In [ ]:
result_files = glob.glob('results/sa/claude-3-5-sonnet-0shot-*_100.csv')

aggregated_results = []

for file in result_files:
    # Extract modification type from filename
    print(file)
    mod_type = file.split('-')[-1].replace('.csv','')
    
    # Read results file
    df = pd.read_csv(file)

    # Calculate accuracies
    original_correct = (df['original_pred'] == df['original_label']).sum()
    modified_correct = (df['modified_pred'] == df['modified_label']).sum()
    total = len(df)

    original_acc = original_correct / total
    modified_acc = modified_correct / total
    
    # Calculate the difference between original_res and modified_res
    difference = -round(original_acc - modified_acc, 2)
    
    # Calculate percentage difference with respect to total samples
    pct_difference = -round((original_correct - modified_correct) / original_correct * 100, 2)
    
    # Perform t-test between original and modified predictions
    t_stat, p_value = stats.ttest_ind(
        (df['original_pred'] == df['original_label']).astype(float),
        (df['modified_pred'] == df['modified_label']).astype(float)
    )
    
    aggregated_results.append({
        'task': 'dialogue_contradiction_detection',
        'modification': mod_type,
        'original_res': round(original_acc, 2),
        'modified_res': round(modified_acc, 2),
        'difference': difference,  # Difference in accuracy
        'pct_difference': pct_difference,  # Percentage difference relative to total samples
        'p_value': p_value  # Add p-value from t-test
    })

# Create final results dataframe
results_df = pd.DataFrame(aggregated_results)

# Sort the results based on modification_name
modification_name = ['temporal_bias_100', 'geographical_bias_100','length_bias_100', 'typo_bias_100', 'capitalization_100', 'punctuation_100', 'derivation_100', 'compound_word_100','active_to_passive_100','grammatical_role_100', 'coordinating_conjunction_100', 'concept_replacement_100','negation_100','discourse_100','sentiment_100','casual_100', 'dialectal_100']
results_df['modification'] = pd.Categorical(results_df['modification'], categories=modification_name, ordered=True)
results_df = results_df.sort_values(by='modification')

# Calculate averages across all modifications
avg_original = results_df['original_res'].mean()
avg_modified = results_df['modified_res'].mean()
avg_difference = avg_original - avg_modified
avg_pct_difference = results_df['pct_difference'].mean()

# Add averages as a new row
results_df.loc[len(results_df)] = {
    'task': 'dialogue_contradiction_detection',
    'modification': 'average',
    'original_res': round(avg_original, 2),
    'modified_res': round(avg_modified, 2),
    'difference': -round(avg_difference, 2),
    'pct_difference': round(avg_pct_difference, 2),
    'p_value': None  # No p-value for average row
}

print("\n")
results_df.to_csv('results/sa/claude-3-5-sonnet-DP.csv')

# Apply styling to highlight rows where original_res > modified_res and significant p-values
def highlight_drops_and_significance(row):
    colors = [''] * len(row)
    if row['original_res'] > row['modified_res']:
        colors = ['background-color: red'] * len(row)
        # If p-value < 0.05, add bold text
        if 'p_value' in row and row['p_value'] is not None and row['p_value'] < 0.05:
            colors = ['background-color: red; font-weight: bold'] * len(row)
    return colors

results_df.round(2).style.apply(highlight_drops_and_significance, axis=1)


In [ ]:
# Load results from different models
gpt4_df = pd.read_csv('results/sa/claude-3-5-sonnet-0shot-sst2.csv')
claude_df = pd.read_csv('results/sa/claude-3-5-sonnet-0shot-sst2.csv')
mixtral_df = pd.read_csv('results/sa/mixtral-8x22b-sst2.csv')

# Calculate accuracy between predictions and labels
gpt4_acc = (gpt4_df['pred'] == gpt4_df['label']).mean()
claude_acc = (claude_df['pred'] == claude_df['label']).mean()
mixtral_acc = (mixtral_df['pred'] == mixtral_df['label']).mean()
# Calculate average accuracy for each model
print(f"GPT-4 Average Accuracy: {gpt4_acc:.2%}")
print(f"Claude-3.5 Average Accuracy: {claude_acc:.2%}")
print(f"Mixtral Average Accuracy: {mixtral_acc:.2%}")

# Create comparison dataframe
comparison_df = pd.DataFrame({
    'Model': ['GPT-4', 'Claude-3.5', 'Mixtral'],
    'Accuracy': [gpt4_acc, claude_acc, mixtral_acc]
})

# Style the dataframe
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

styled_df = comparison_df.style.apply(highlight_max, subset=['Accuracy'])
styled_df
